In [1]:
!pip install mmcv
import pandas as pd
import os
import json
import pickle
import numpy as np
import cv2
import mmcv
from PIL import Image, ImageDraw

     |████████████████████████████████| 390 kB 14.5 MB/s 
     |████████████████████████████████| 185 kB 60.5 MB/s 
  Created wheel for mmcv: filename=mmcv-1.3.17-py2.py3-none-any.whl size=573414 sha256=68657f371eff59afd86a97be6deae929c8fb0ad9c9cda3891a53db23a0c7dd9e
  Stored in directory: /root/.cache/pip/wheels/4d/44/de/6831f4485611ea75714e7e8fad8d12b7f04933481c141f830b
Successfully built mmcv
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
posedir = "/content/drive/MyDrive/projects/pose_detection/results/pose_detection"
frdir = "/content/drive/MyDrive/projects/pose_detection/results/face_recognition"
combined_results_dir = "/content/drive/MyDrive/projects/pose_detection/results/combined_fr_pd"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
poselist_paths = os.listdir(posedir)
fr_results_files = os.listdir(frdir)

In [4]:
# given the pose list and the dataframe with candidate face positions
# get only the candidate's poses
def get_candidate_poses(poselist, fr_path):

    l = pd.read_pickle(fr_path)
    df = pd.DataFrame.from_dict(l, orient = "index", columns = ["x0", "y0", "x1", "y1"])

    candidate_pose_landmarks = []

    for i, frame in enumerate(poselist):

        candidate_in_frame = []

        for person_keypoints in frame:

            pose_face_x = person_keypoints[0:2][0]
            pose_face_y = person_keypoints[0:2][1]

            if i in df.index:
                in_x = df.loc[[i]]['x1'].item() >= pose_face_x >= df.loc[[i]]['x0'].item()
                in_y = df.loc[[i]]['y1'].item() >= pose_face_y >= df.loc[[i]]['y0'].item()

                # is the pose face the candidate?
                is_candidate = in_x & in_y

                if is_candidate:
                    candidate_in_frame = person_keypoints

        candidate_pose_landmarks.append(candidate_in_frame)

    return(candidate_pose_landmarks)


#
def fill_cand_array(cand_array, poselist):

    for i, row in enumerate(cand_array):

        if i == (len(cand_array)-1):
            break

        # print(i)
        # print(row)
         
        # If the current row is nan
        if all(np.isnan(cand_array[i])):
            # And the next row is not nan
            if all(np.isnan(cand_array[i+1])) == False:

                match_found = True
                d=0

                # backfill
                while (match_found == True) & (i-d>=0) & all(np.isnan(cand_array[i-d])):

                    match_found = False

                    for j in poselist[i-d]:

                        frame_distance = np.abs(np.nanmean(np.absolute(cand_array[i+1-d]-np.array(j))))

                        if frame_distance <= 5:
                            cand_array[i-d] = np.array(j)

                            match_found = True
                            d += 1

                        del frame_distance

            # And the previous row is not nan
            # forward fill
            # here we only need to do it once rather than also retroactively for the rows before
            # because we are iterating over all rows in a forward manner anyway
            elif all(np.isnan(cand_array[i-1])) == False:

                for j in poselist[i]:

                        frame_distance = np.abs(np.nanmean(np.absolute(cand_array[i-1]-np.array(j))))

                        if frame_distance <= 5:
                            cand_array[i] = np.array(j)

                        del frame_distance

    return(cand_array)


In [5]:
# Infer the face recognition path from the pose path
# Replace the first underscore with a slash to make the year a directory
fr_paths = [f.replace(".pkl", "_cand_boxes.pkl") for f in poselist_paths]
# Add directory
fr_paths = [frdir + '/' + p for p in fr_paths]
pose_paths = [posedir + '/' + p for p in poselist_paths]

# If the inferred fr path exists, make it a dictionary key
# with the corresponding pose path as the value
ref_dict = {}
for i, name in enumerate(pose_paths):
    if os.path.exists(fr_paths[i]):
        ref_dict[fr_paths[i]] = name

In [6]:
combined_results_dir = "/content/drive/MyDrive/projects/pose_detection/results/combined_fr_pd"

In [8]:
counter = 0
for k in ref_dict:

    # Check if file has already been done, if so, skip
    output_path = combined_results_dir + "/" + os.path.split(os.path.split(k)[0])[1] + "_" + os.path.basename(k).replace("_cand_boxes.pkl", "_landmarks.csv")
    if os.path.exists(output_path)==False:

        print(ref_dict[k])
        # Load the poselist
        posel = pickle.load(open( ref_dict[k], "rb" ))
        # Combine the poselist with the facial recognition
        # to get only the candidate's poses
        fr_path = k
        candidate_poses = get_candidate_poses(posel, fr_path)
        # Convert to pd dataframe and then np array
        df_candidate = pd.DataFrame(candidate_poses)
        cand_array = np.array(df_candidate)
        # Fill out missing rows in the array
        # if they are inferred to belong to the candidate
        cand_array = fill_cand_array(cand_array, posel)

        # Convert the numpy to a pandas dataframe...
        colnamesx = ["x" + str(i) for i in range(25)]
        colnamesy = ["y" + str(i) for i in range(25)]
        colnames = [None]*(len(colnamesx)+len(colnamesy))
        colnames[::2] = colnamesx
        colnames[1::2] = colnamesy
        if cand_array.size == 0:
            cand_df = pd.DataFrame(columns = colnames)
        else:
            cand_df = pd.DataFrame(cand_array, columns = colnames)
        # ... and save it
        cand_df.to_csv(output_path, index=False)
        print(counter)
        counter += 1

        #if counter == 5:
        #  break

/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_NY_WOERNER_POST_STAR_ENDORSEMENT.pkl
0
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_NY_WOERNER_THESE_ADS.pkl
1
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_LYERLY_WE_ALL_DO_BETTER.pkl
2
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_MEYERS_CLEAN_WATER.pkl
3
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_RODRIGUEZ_A_WORKING_MOM.pkl
4
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_SCHIERL_WISCONSIN_NEEDS_LEADERS.pkl
5
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_SCHUTZ_I_LIVED_IN_RACINE_COUNTY.pkl
6
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STAMBLY_WI_STEFFEN_IT_WAS_SCARY.pkl
7
/content/drive/MyDrive/projects/pose_detection/results/pose_dete

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice


10
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_AR_COMSTOCK_IMPRESSED_WITH_JON_45.pkl
11
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_AR_WING_SAVE_THE_STATE_OF_ARKANSAS.pkl
12
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_AZ_HERNANDEZ_THE_PANDEMIC_SP.pkl


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice


13
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_AZ_LYONS_WANTED_TO_PROTECT_PEOPLE.pkl
14
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_ANDRADE_HOMETOWN_CONSERVATIVE_15.pkl
15
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_BARREIRO_HAS_THE_EXPERIENCE.pkl
16
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_BARRIOS_DICTATORS_15_SP.pkl
17
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_BARRIOS_VOTE_FOR_PEDRO_15_SP.pkl
18
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_BARRIOS_FIGHT_FOR_YOU_SP.pkl
19
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_STREP_FL_CORTES_A_LEADER_WE_CAN_TRUST_SP.pkl


...

796
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MO_SCHMITT_FIGHTING_BACK_AGAINST_VIOLENT_CRIME.pkl
797
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MO_SCHMITT_FIGHTING_VIOLENT_CRIME.pkl
798
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_GRAYBILL_NEVER_BACK_DOWN.pkl
799
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_GRAYBILL_NEVER_BACKED_DOWN_FROM_BIG_FIGHTS.pkl
800
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_GRAYBILL_THERES_A_LOT_OF_NOISE_OUT_THERE.pkl
801
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_KNUDSEN_ENDORSED_15.pkl
802
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_KNUDSEN_TELLING_LIES_15.pkl
803
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_MT_KNUDSEN_VIOLENT_CRIMES_UP_36_PERCENT.pkl

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice


816
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_UT_REYES_THE_MONTH.pkl


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice


817
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WA_LARKIN_ENFORCE_THE_LAW.pkl
818
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WA_ROGERS_PUBLIC_SAFETY.pkl
819
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WA_LARKIN_MIKE_SOLAN.pkl
820
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WV_MORRISEY_HAS_A_TOUGH_INDEPENDENT15.pkl
821
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WV_MORRISEY_HAS_A_TOUGH_INDEPENDENT_ATTORNEY_GENERAL.pkl
822
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WV_PETSONK_EDUCATIONAL_CRISIS.pkl
823
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_ATGEN_WV_PETSONK_THE_PEOPLES_LAWYER.pkl
824
/content/drive/MyDrive/projects/pose_detection/results/pose_detection/2020_DELG_WV_CAPITO_REMOTE_WORK.pkl
825
/content/drive/MyDrive/projects/pose_dete